In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import os
import csv
import pandas as pd
import numpy as np
import requests
import time
import json
from pandas import json_normalize
from scipy.stats import linregress
from pprint import pprint

In [2]:
# Set the API base URL 
url = f"https://data.austintexas.gov/resource/wter-evkm.json"

# Create URL for GET requests
animal_data = requests.get(url).json()

# Convert the API to a DataFrame and display data
animal_df = pd.DataFrame.from_dict(json_normalize(animal_data))
animal_df.tail(5)

,animal_id,datetime,datetime2,found_location,intake_type,intake_condition,animal_type,sex_upon_intake,age_upon_intake,breed,color,name
995,A664268,2013-10-01T12:05:00.000,2013-10-01T12:05:00.000,Austin (TX),Stray,Normal,Cat,Intact Female,3 years,Russian Blue Mix,Blue,*Kochka
996,A656696,2013-10-13T12:52:00.000,2013-10-13T12:52:00.000,Austin (TX),Owner Surrender,Normal,Cat,Spayed Female,2 years,Domestic Shorthair Mix,Brown Tabby,*Angel
997,A665285,2013-10-15T18:19:00.000,2013-10-15T18:19:00.000,4807 Airport Blvd in Austin (TX),Stray,Normal,Cat,Intact Female,5 months,Domestic Shorthair Mix,Torbie,Tabby
998,A665185,2013-10-14T18:32:00.000,2013-10-14T18:32:00.000,Hwy 183/Burnet Rd in Austin (TX),Stray,Normal,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Brown Tabby/White,*Vince
999,A665572,2013-10-19T14:09:00.000,2013-10-19T14:09:00.000,900 Broken Feather Trl in Pflugerville (TX),Stray,Normal,Dog,Intact Male,1 year,Great Pyrenees Mix,White,NaN


In [5]:
## JSON results are limited and will not provide 2022 information.
## Switching from JSON to CSV.

In [6]:
# Path to collect CSV data from folder
#animal_data = os.path.join("C://Users//Junk//Desktop//My Project Folder//Austin_Animal_Center_Intakes.csv")
animal_data = os.path.join("Project_1", "Resources", "Animal_Intakes.csv")

# Convert CSV to DataFrame
animal_df = pd.read_csv("C://Users//Junk//Desktop//My Project Folder//Austin_Animal_Center_Intakes.csv") 
animal_df.tail(5)

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
153561,A775309,Snowball,06/27/2018 06:16:00 PM,June 2018,Austin (TX),Owner Surrender,Normal,Cat,Spayed Female,2 years,Turkish Angora,White
153562,A760677,NaN,10/22/2017 01:58:00 PM,October 2017,Thomaswood Lane And Lenape Trail in Austin (TX),Stray,Normal,Cat,Intact Male,2 years,Domestic Shorthair Mix,Blue/White
153563,A811255,NaN,12/31/2019 08:10:00 AM,December 2019,12034 Research Blvd in Austin (TX),Stray,Injured,Cat,Intact Female,5 months,Domestic Shorthair,Black
153564,A763156,NaN,12/04/2017 11:42:00 AM,December 2017,1009 West Dittmar Road in Austin (TX),Stray,Normal,Dog,Intact Male,2 years,Miniature Poodle/Cocker Spaniel,White
153565,A855904,*Julissa,04/25/2022 03:22:00 PM,April 2022,9109 Marsh Dr in Austin (TX),Stray,Normal,Dog,Intact Female,9 months,Labrador Retriever Mix,Red


In [19]:
# Creating a DataFrame copy to reduce to essential columns
revised_df= animal_df[["Animal ID", "MonthYear", "Intake Type", "Animal Type", "Breed"]].copy()
revised_df.tail(5)
revised_df.dtypes

Animal ID      object
MonthYear      object
Intake Type    object
Animal Type    object
Breed          object
dtype: object

In [34]:
# Rename column and split a DataFrame column
#revised_df.rename(columns = {"MonthYear":"Month Year"}, inplace = True)
#revised_df["Month Year"].str.partition(' ')[[Month, Year]]
#revised_df[["Month", "Year"]] = revised_df."Month Year".str.split(" ", expand = True)

revised_df[["Month","Year"]] = revised_df["Month Year"].str.split(" ",expand=True)
revised_df.tail(5)

,Animal ID,Month Year,Intake Type,Animal Type,Breed,Month,Year
153561,A775309,June 2018,Owner Surrender,Cat,Turkish Angora,June,2018
153562,A760677,October 2017,Stray,Cat,Domestic Shorthair Mix,October,2017
153563,A811255,December 2019,Stray,Cat,Domestic Shorthair,December,2019
153564,A763156,December 2017,Stray,Dog,Miniature Poodle/Cocker Spaniel,December,2017
153565,A855904,April 2022,Stray,Dog,Labrador Retriever Mix,April,2022


In [35]:
# Cleaning and sorting the data
split_df = revised_df[["Animal ID", "Month", "Year", "Intake Type", "Animal Type", "Breed"]].copy()
split_df.tail(5)

,Animal ID,Month,Year,Intake Type,Animal Type,Breed
153561,A775309,June,2018,Owner Surrender,Cat,Turkish Angora
153562,A760677,October,2017,Stray,Cat,Domestic Shorthair Mix
153563,A811255,December,2019,Stray,Cat,Domestic Shorthair
153564,A763156,December,2017,Stray,Dog,Miniature Poodle/Cocker Spaniel
153565,A855904,April,2022,Stray,Dog,Labrador Retriever Mix


In [25]:
Intake_Counts = animal_df["Intake Type"].value_counts()
Intake_Counts

Stray                 105218
Owner Surrender        31799
Public Assist           9176
Wildlife                5905
Abandoned               1209
Euthanasia Request       259
Name: Intake Type, dtype: int64

In [10]:
Intake_Counts = animal_df["Animal Type"].value_counts()
Intake_Counts

Dog          85516
Cat          59115
Other         8145
Bird           764
Livestock       26
Name: Animal Type, dtype: int64